In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

pd.options.display.float_format = '${:,.2f}'.format

In [ ]:
## -- Read Data
PATH_TO_DATA = 'Airline/airline-passengers.csv'
df = pd.read_csv(PATH_TO_DATA, dtype=str)

## -- Preprocessing
df['Month'] = pd.to_datetime(df['Month'])
df['Passengers'] = df['Passengers'].astype('int32')

In [ ]:
## -- Display sample data
df.head(12)

In [ ]:
## -- Create a time-series plot to visualize the data
plt.figure(figsize=(15, 6))
plt.plot(df['Month'], df['Passengers'])
plt.title('Airline Passengers')
plt.xlabel('Month')
plt.ylabel('No. of Passengers')
plt.show()

In [ ]:
## -- Set Month as Index
df.set_index('Month', inplace=True)

## -- Seasonal Decomposition
result = seasonal_decompose(df['Passengers'], model='multiplicative')

In [ ]:
## -- Plot the original time series
plt.figure(figsize=(12, 8))
plt.subplot(4, 1, 1)
plt.plot(df['Passengers'], label='Original')
plt.legend(loc='upper left')

## -- Plot the trend component
plt.subplot(4, 1, 2)
plt.plot(result.trend, label='Trend')
plt.legend(loc='upper left')

## -- Plot the seasonal component
plt.subplot(4, 1, 3)
plt.plot(result.seasonal, label='Seasonal')
plt.legend(loc='upper left')

## -- Plot the residual component
plt.subplot(4, 1, 4)
plt.plot(result.resid, label='Residual')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

## Prophet

In [ ]:
from prophet import Prophet
import cmdstanpy
cmdstanpy.install_cmdstan(compiler=True)

In [ ]:
## -- Reset index to get Month back to Column
df = df.reset_index()

## -- Rename columns to the specific name as required by Prophet
df = df.rename(columns={'Month': 'ds', 'Passengers': 'y'})

In [ ]:
## -- Fit the Prophet model
model = Prophet()
model.fit(df)

In [ ]:
## -- Make future date predictions
future = model.make_future_dataframe(periods=24, freq='M')
forecast = model.predict(future)

In [ ]:
## -- Visualize the forecast
fig = model.plot(forecast)
## -- Customize the plot, change the color of the future period
plt.fill_between(future['ds'][len(df):], forecast['yhat_lower'][len(df):], forecast['yhat_upper'][len(df):], \
                 color='orange', alpha=0.3, label='Future Period')
plt.title('Airline Passengers - Forecasting the next 2 years')
plt.xlabel('Month')
plt.ylabel('No. of Passengers')
plt.show()

## Evaluation

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Evaluate the predictions
y_true = df['y'].values
y_pred = forecast['yhat'].values[:len(df)]

# Calculate evaluation metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)

print(f'Mean Absolute Error (MAE): {mae:,.2f}')
print(f'Mean Squared Error (MSE): {mse:,.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:,.2f}')

In [ ]:
import prophet
prophet.__version__